In [227]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import RandomizedSearchCV
from ImbalancedLearningRegression import smote
from sklearn.metrics import r2_score, mean_squared_error

In [179]:
bikes_test = pd.read_excel('bike_test.xlsx')
bikes_train = pd.read_excel('bike_train.xlsx')

In [180]:
bikes_test.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed
0,12000,2012-05-20,2,1,5,4,0,0,0,1,0.52,0.5000,0.68,0.0896
1,12001,2012-05-20,2,1,5,5,0,0,0,1,0.50,0.4848,0.72,0.1045
2,12002,2012-05-20,2,1,5,6,0,0,0,1,0.50,0.4848,0.63,0.1343
3,12003,2012-05-20,2,1,5,7,0,0,0,1,0.52,0.5000,0.68,0.1940
4,12004,2012-05-20,2,1,5,8,0,0,0,1,0.56,0.5303,0.56,0.1642


In [181]:
bikes_train.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


In [182]:
#in case i need to work with the dataframe so i don't touch the original
bikes_test_copy = bikes_test.copy()
bikes_train_copy = bikes_train.copy()

In [183]:
bikes_test_copy.shape, bikes_train_copy.shape

((5380, 14), (11999, 17))

In [184]:
bikes_train_copy.columns.unique()

Index(['instant', 'dteday', 'season', 'yr', 'mnth', 'hr', 'holiday', 'weekday',
       'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed',
       'casual', 'registered', 'cnt'],
      dtype='object')

In [185]:
bikes_test_copy.columns.unique()

Index(['instant', 'dteday', 'season', 'yr', 'mnth', 'hr', 'holiday', 'weekday',
       'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed'],
      dtype='object')

In [186]:
bikes_train_copy.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


In [187]:
bikes_train_copy.info()

#we can see we don't have any null values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11999 entries, 0 to 11998
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   instant     11999 non-null  int64         
 1   dteday      11999 non-null  datetime64[ns]
 2   season      11999 non-null  int64         
 3   yr          11999 non-null  int64         
 4   mnth        11999 non-null  int64         
 5   hr          11999 non-null  int64         
 6   holiday     11999 non-null  int64         
 7   weekday     11999 non-null  int64         
 8   workingday  11999 non-null  int64         
 9   weathersit  11999 non-null  int64         
 10  temp        11999 non-null  float64       
 11  atemp       11999 non-null  float64       
 12  hum         11999 non-null  float64       
 13  windspeed   11999 non-null  float64       
 14  casual      11999 non-null  int64         
 15  registered  11999 non-null  int64         
 16  cnt         11999 non-

In [188]:
bikes_train_copy.nunique()
#we can see 4 seasons, 12 months, 7 weekdays
#nothing out of the oridinary

instant       11999
dteday          506
season            4
yr                2
mnth             12
hr               24
holiday           2
weekday           7
workingday        2
weathersit        4
temp             48
atemp            65
hum              88
windspeed        30
casual          287
registered      623
cnt             713
dtype: int64

In [189]:
bikes_train_copy.isna().any()
#like we said before no missing values

instant       False
dteday        False
season        False
yr            False
mnth          False
hr            False
holiday       False
weekday       False
workingday    False
weathersit    False
temp          False
atemp         False
hum           False
windspeed     False
casual        False
registered    False
cnt           False
dtype: bool

In [190]:
#a piece of code to pair up correlation i found
#it transforms every value to abolute value, meaning negative to positive
#and then uses the unstack method to sort of transpose rows and columns
#it just works
corr = bikes_train_copy.corr()
corr_unstack = corr.abs().unstack()
corr_unstack.sort_values(ascending = False)[16:27:2]

atemp       temp      0.991785
registered  cnt       0.968540
season      mnth      0.865268
instant     yr        0.777284
cnt         casual    0.702414
registered  casual    0.503185
dtype: float64

Here we can see that atemp are temp and closely correlated so we can drop atemp since it's irrelevant
and it will only create noise

The instant column is just the ID number the daily report gets so that one has to be dropped

We can also drop instant because it's just an identifier

We can drop yr because it's irrelevant to people hiring rental bikes

Finally we can also drop the dteday column because they are the same every year and has nothing to do
with the issue, the only relevant data related to date is the day of the week,
if it is a working day or not and if it's a holiday

In [191]:
#the following columns have a set number of values so they can be turned into category types
#we can just copy the names in the results of the cell above
columns_to_category = ["dteday", "season", "yr", "mnth", "hr", "holiday", "weekday", "workingday", "weathersit"]
#the following fields after weathersit, like temp and hr, have too many variables
#so they can't be categories
for column in columns_to_category:
    bikes_train_copy[column] = bikes_train_copy[column].astype('category')

In [192]:
bikes_train_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11999 entries, 0 to 11998
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   instant     11999 non-null  int64   
 1   dteday      11999 non-null  category
 2   season      11999 non-null  category
 3   yr          11999 non-null  category
 4   mnth        11999 non-null  category
 5   hr          11999 non-null  category
 6   holiday     11999 non-null  category
 7   weekday     11999 non-null  category
 8   workingday  11999 non-null  category
 9   weathersit  11999 non-null  category
 10  temp        11999 non-null  float64 
 11  atemp       11999 non-null  float64 
 12  hum         11999 non-null  float64 
 13  windspeed   11999 non-null  float64 
 14  casual      11999 non-null  int64   
 15  registered  11999 non-null  int64   
 16  cnt         11999 non-null  int64   
dtypes: category(9), float64(4), int64(4)
memory usage: 889.6 KB


In [193]:
#it would be nice if the columns were easily recognizable
#it's a good idea to rename them when neccesary
bikes_train_copy.rename(columns= {'dteday':'date', 'yr':'year', 'mnth':'month', 'hr': 'hour', 'weathersit': 'weather', 'temp': 'temperature_C', 'atemp': 'temperature_wind-chill_factor_C',  'hum':'humidity', 'cnt':'count'}, inplace= True)
bikes_train_copy.head()

,instant,date,season,year,month,hour,holiday,weekday,workingday,weather,temperature_C,temperature_wind-chill_factor_C,humidity,windspeed,casual,registered,count
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


In [194]:
#here we are dropping the columns mentioned above
bikes_train_copy = bikes_train_copy.drop(columns=['instant', 'temperature_wind-chill_factor_C', 'date', 'year'],axis=1)

In [195]:
bikes_train_copy.head()

,season,month,hour,holiday,weekday,workingday,weather,temperature_C,humidity,windspeed,casual,registered,count
0,1,1,0,0,6,0,1,0.24,0.81,0.0,3,13,16
1,1,1,1,0,6,0,1,0.22,0.80,0.0,8,32,40
2,1,1,2,0,6,0,1,0.22,0.80,0.0,5,27,32
3,1,1,3,0,6,0,1,0.24,0.75,0.0,3,10,13
4,1,1,4,0,6,0,1,0.24,0.75,0.0,0,1,1


In [196]:
#now we can transform each column, season, month, etc.
#and split them into their variables
#e.g weekday[0, 1, 2, 3, 4, 5, 6] into weekday_1, weekday_2, etc.
#and it will have a 1 when the state is true and 0 for the rest
categories_to_split = ['season', 'month', 'hour', 'holiday', 'weekday', 'workingday', 'weather']
bikes_train_copy = pd.get_dummies(bikes_train_copy, columns = categories_to_split, drop_first= True)
bikes_train_copy.head()

,temperature_C,humidity,windspeed,casual,registered,count,season_2,season_3,season_4,month_2,...,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,workingday_1,weather_2,weather_3,weather_4
0,0.24,0.81,0.0,3,13,16,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0.22,0.80,0.0,8,32,40,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0.22,0.80,0.0,5,27,32,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,0.24,0.75,0.0,3,10,13,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,0.24,0.75,0.0,0,1,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [197]:
#Using ImbalancedLinearRegression.smote we oversample the dataset creating new values based on
#current ones to have a bigger sample size
#we don't have enough samples to make a proper prediction
#this takes the "y" value, "count" in this case and
#using its neighbours generate new samples

data_bikes = smote(
    data = bikes_train_copy,
    y = 'count', #target column
    k = 5, #number of neighbours to consider
    samp_method = 'extreme',
    rel_thres = 0.9, #to determine what is considered rare
    rel_method = 'auto', #manual requires proper knowledge of how the function works, auto for now
    rel_xtrm_type = 'high', #we use high so rare values appearing rarely is considered normal
    rel_coef = 0.9
)

r_index: 100%|##########| 572/572 [00:02<00:00, 212.40it/s]


In [199]:
#i like to make copies
#in case i make any mistakes, i'm not waiting for 33 minutes again
data_bikes_copy = data_bikes.copy()

In [213]:
#X is going to be the dataset minus the 3 columns with rentals, casual, registered and count
#y is going to be those casual and registered, not count since it's just the sum of the previous two
X = data_bikes_copy.drop(columns=['casual', 'registered', 'count'])
y = data_bikes_copy[['casual', 'registered']]
X.shape, y.shape

((21157, 51), (21157, 2))

In [214]:
#here we just split the sets like always
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

trying random forest

In [215]:
#we are just declaring the model
random_forest_model = RandomForestRegressor(random_state=42)

In [216]:
#we just use fit and predict like in any model
random_forest_model.fit(X_train, y_train)
random_forest_model_prediction = random_forest_model.predict(X_test)

In [217]:
#we get each score and multiply the r2 score so it's a number between 0 and 100
print('R2 Score: ', r2_score(y_test, random_forest_model_prediction)*100)
print('RMSE: ', np.sqrt(mean_squared_error(y_test, random_forest_model_prediction)))

R2 Score:  86.84871783988719
RMSE:  49.166320302181944


In [218]:
random_forest_model_prediction_train = random_forest_model.predict(X_train)

In [219]:
print('R2 Score: ', r2_score(y_train, random_forest_model_prediction_train)*100)
print('RMSE: ', np.sqrt(mean_squared_error(y_train, random_forest_model_prediction_train)))

R2 Score:  98.12380445476474
RMSE:  18.314613998411605


In [225]:
#standard parameters i found
#not much to explain here
parameters={'n_estimators':[int(x) for x in np.linspace(50,2000,200)],
            'max_depth':[int(x) for x in np.linspace(1,50,30)],
            'criterion':["mse", "mae"],
            'min_samples_split': [int(x) for x in np.linspace(1,50,30)],
            'min_samples_leaf': [int(x) for x in np.linspace(1,50,30)]}

In [228]:
#parameter tuning using randomizedsearchcv
#the parameters are standard ones i found
#not much to explain here
#it's useful to increase the model's generalizability
random_forest_model_tuned=RandomizedSearchCV(random_forest_model,parameters,cv=5,n_iter=30,n_jobs=-1,verbose=5,random_state=2)
random_forest_model_tuned.fit(X_train,y_train)
random_forest_model_tuned.best_score_

Fitting 5 folds for each of 30 candidates, totalling 150 fits


In [ ]:
random_forest_model_modified=random_forest_model_tuned.best_estimator_
random_forest_model_modified

In [ ]:
# r2_score on test data
random_forest_model_modified.fit(X_train,y_train)
y_prediction_modified_01=random_forest_model_modified.predict(X_test)

In [ ]:
print('Testing R2 Score: ', r2_score(y_test, y_prediction_modified_01)*100)
print('Testing RMSE: ', np.sqrt(mean_squared_error(y_test, y_prediction_modified_01)))

In [ ]:
y_prediction_modified_02=random_forest_model_modified.predict(X_train)

In [ ]:
print('Training R2 Score: ', r2_score(y_train, y_prediction_modified_02)*100)
print('Training RMSE: ', np.sqrt(mean_squared_error(y_train, y_prediction_modified_02)))

In [221]:
#the same method we use for any other regressor
# gradient_boost_model = MultiOutputRegressor(GradientBoostingRegressor(random_state=2))

# gradient_boost_model.fit(X_train, y_train)
# gradient_boost_model_prediction = gradient_boost_model.predict(X_test)

In [222]:
# print('R2 Score: ', r2_score(y_test, gradient_boost_model_prediction)*100)
# print('RMSE: ', np.sqrt(mean_squared_error(y_test, gradient_boost_model_prediction)))

R2 Score:  77.05079765650758
RMSE:  66.06101778881961


In [223]:
# gradient_boost_model_prediction_train = gradient_boost_model.predict(X_train)

In [224]:
# print('R2 Score: ', r2_score(y_train, gradient_boost_model_prediction_train)*100)
# print('RMSE: ', np.sqrt(mean_squared_error(y_train, gradient_boost_model_prediction_train)))

R2 Score:  77.43693261109452
RMSE:  64.64463856480633
